#### Running model on one sample from the Desert dataset. Sample "mgs283966" obtained from MG-RAST. Smallest sample in dataset: ~1.2E9 bp, ~7.2E6 sequences

Citation:

Finkel OM, Delmont TO, Post AF, Belkin S. Metagenomic Signatures of Bacterial Adaptation to Life in the Phyllosphere of a Salt-Secreting Desert Tree. Appl Environ Microbiol. 2016 Apr 18;82(9):2854-61.

#### When I did this, there were no hits to the plasmids, I decided to try another sample from the Urban dataset and from NCBI SRA. Hsu_etal_2016_SRA

In [1]:
import pandas as pd
import numpy as np
import csv

from Bio import Entrez
from time import sleep

import matplotlib.pyplot as plt
import seaborn as sns

import gensim
import pickle

import os
import gzip

In [2]:
model = pickle.load(open("w5_model.p", "rb"))
model_vectors = model.wv

In [3]:
#processed to only include sequences, used the '5. Host DNA contamination removal' version of the sample from MG-RAST

desert_sample = open('mgm4620735.3.299.screen.passed.fna', 'r').readlines()
desert_sample = [seq.rstrip().upper() for seq in desert_sample[1::2]]

In [13]:
len(desert_sample)

5670854

Here is where I attempt to iterate over all of the files in the hsu_urban directory. Also need to be unzipped and the files are large, kept read 1 and 2

In [4]:
# urban_sample = []
# directory = os.fsencode('hsu_urban')

# for file in os.listdir(directory):
#     sample = []
#     with gzip.open(file,'rb') as f:        
#         for line in f:
#             sample.append(line.decode("utf-8"))
#             sample = [i.rstrip() for i in sample[1::4]]
#         f.close()
#     urban_sample += sample

#### This process is similar to how the model was trained, but to save memory only k-mers that have a match in the model are kept.

In [5]:
sample = desert_sample
vectorized_sample = []
kmer_len = 20

for ind,sequence in enumerate(sample):
    print('\r' + 'Tokenizing sequence: ' + str(ind+1) + ' of ' + str(len(sample)) + 
                  ', or ' + str(round((ind/len(sample))*100, 4)) + '% done', end='')
    for j in range(kmer_len):
        counter = 0
        num_matches = 0
        try:
            vectorized_sequence = []
            while counter+kmer_len < len(sequence):
                try:
                    model_vectors.similar_by_word(sequence[j+counter:j+counter+kmer_len])
                    if num_matches == 0:
                        vectorized_sequence = model_vectors[sequence[j+counter:j+counter+kmer_len]]
                    else:
                        vectorized_sequence = np.add(vectorized_sequence, model_vectors[sequence[j+counter:j+counter+kmer_len]])
                except:
                    pass
                counter += kmer_len
                num_matches += 1
            vectorized_sample.append(vectorized_sequence/num_matches)
        except:
            pass

Tokenizing sequence: 219 of 5670854, or 0.0038% done

Tokenizing sequence: 457 of 5670854, or 0.008% done

Tokenizing sequence: 5220826 of 5670854, or 92.0642% done

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Tokenizing sequence: 5367140 of 5670854, or 94.6443% done

Tokenizing sequence: 5367415 of 5670854, or 94.6491% done

Tokenizing sequence: 5367563 of 5670854, or 94.6517% done

Tokenizing sequence: 5368474 of 5670854, or 94.6678% done

Tokenizing sequence: 5670854 of 5670854, or 100.0% donene

In [11]:
desert_df = pd.DataFrame(vectorized_sample)
desert_df.to_csv("desert_sample.csv")

In [12]:
len(desert_df)

2663